In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
import config

config.root_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.insert(0, config.root_path)

In [3]:
import tensorflow as tf

from tensorflow.python import keras
import json
import pandas as pd
import numpy as np
import os

from src.encoders.context_encoder_ldabert_2 import ContextEncoder
from src.dataset.ldabert_2 import LDABERT2Dataset

## LDA BERT 2 Dataset

In [4]:
dataset_type = "clinical"

In [9]:
dataset = LDABERT2Dataset(dataset_type=dataset_type,
                           pct_data=0.005,
                          max_seq_length=128,
                           max_segment_length=300,
                           augment_pct=0)

sentences, tokenized_sentences, labels = dataset.process()

# vectors_path = '../data/clinical_vectors/lda_bert_{}_{}.pkl'.format(dataset_type, len(sentences))
vectors_path = '../../data/lda_bert_2/{}/{}_{}.pkl'.format(dataset_type, 
                                                        dataset.pct_data, 
                                                        dataset.augment_pct)

saved_vectors, saved_labels, saved_sentences, saved_tokenized_sentences = dataset.get_saved_vectors(vectors_path)

if len(saved_vectors) == 0:
    saved_vectors, saved_labels, saved_sentences, saved_tokenized_sentences = dataset.create_vectors(vectors_path)

something went wrong [Errno 2] No such file or directory: 'C:\\Users\\Computer\\Google Drive\\SCHOOL\\PhD\\Code\\context-encoder-v2\\data\\lda_bert_2\\generated_vectors\\clinical\\0.005_0.pkl'
root path C:\Users\Computer\Google Drive\SCHOOL\PhD\Code\context-encoder-v2
Preprocessing raw texts ...
sentences length 130
Preprocessing raw texts. Done!
lda sentences length 130
Getting vector representations for LDA ...
Getting vector representations for LDA. Done!
saving vectors... 130 130 130


In [10]:
left_input, mid_input, right_input = dataset.format_sentences_tri_input_plus(saved_tokenized_sentences)

In [11]:
lda_left_input, lda_mid_input, lda_right_input = dataset.format_sentences_tri_input(saved_vectors)

In [12]:
tokenized_sentences['input_ids'].shape, saved_vectors.shape

(TensorShape([130, 76]), (130, 10))

## Tri Encoder

In [19]:
context_encoder = ContextEncoder(final_dropout=0.5, 
                                 dense_neurons=128, 
                                 max_sentence_length=128, 
                                 bert_trainable=False,
                                 gamma=15)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [20]:
def trim_for_test_input(arr):
    return arr[:5]

def trim_encoded_for_test_input(arr):
    return {
        'input_ids': arr['input_ids'][:5],
        'token_type_ids': arr['token_type_ids'][:5],
        'attention_mask': arr['attention_mask'][:5]
    }

In [21]:
# create smaller test vectors to init model
test_left_input = trim_encoded_for_test_input(left_input)
test_mid_input = trim_encoded_for_test_input(mid_input)
test_right_input = trim_encoded_for_test_input(right_input)
test_lda_left_input = trim_for_test_input(lda_left_input)
test_lda_mid_input = trim_for_test_input(lda_mid_input)
test_lda_right_input = trim_for_test_input(lda_right_input)

In [22]:
context_encoder([test_left_input, test_mid_input, test_right_input, test_lda_left_input, test_lda_mid_input, test_lda_right_input])

<tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[0.6757858 ],
       [0.5165765 ],
       [0.72419494],
       [0.7739043 ],
       [0.58298475]], dtype=float32)>

In [23]:
context_encoder.summary()

Model: "context_encoder_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
tf_bert_model_1 (TFBertModel multiple                  109482240 
_________________________________________________________________
dense_input_left (Dense)     multiple                  99712     
_________________________________________________________________
dense_input_mid (Dense)      multiple                  99712     
_________________________________________________________________
dense_input_right (Dense)    multiple                  99712     
_________________________________________________________________
dense_output (Dense)         multiple                  385       
_________________________________________________________________
final_dropout (Dropout)      multiple                  0         
Total params: 109,781,761
Trainable params: 299,521
Non-trainable params: 109,482,240
_____________________________

In [24]:
checkpoint_filepath = '../models/LDABERT2/simple/clinical-157-0.01-pct-0.01-aug_WT6OE/no-finetune/checkpoint.ckpt'


# try:
context_encoder.load_weights(checkpoint_filepath)
# except:
#     print("No checkpoint available.")

ValueError: Shapes (12, 64, 768) and (768, 768) are incompatible

In [30]:
from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file

print_tensors_in_checkpoint_file(file_name=checkpoint_filepath, tensor_name='', all_tensors=False,
                                     all_tensor_names=True)

tensor: _CHECKPOINTABLE_OBJECT_GRAPH (string) []
tensor: dense_input_1/bias/.ATTRIBUTES/VARIABLE_VALUE (float32) [128]
tensor: dense_input_1/bias/.OPTIMIZER_SLOT/optimizer/m/.ATTRIBUTES/VARIABLE_VALUE (float32) [128]
tensor: dense_input_1/bias/.OPTIMIZER_SLOT/optimizer/v/.ATTRIBUTES/VARIABLE_VALUE (float32) [128]
tensor: dense_input_1/kernel/.ATTRIBUTES/VARIABLE_VALUE (float32) [778, 128]
tensor: dense_input_1/kernel/.OPTIMIZER_SLOT/optimizer/m/.ATTRIBUTES/VARIABLE_VALUE (float32) [778, 128]
tensor: dense_input_1/kernel/.OPTIMIZER_SLOT/optimizer/v/.ATTRIBUTES/VARIABLE_VALUE (float32) [778, 128]
tensor: dense_input_2/bias/.ATTRIBUTES/VARIABLE_VALUE (float32) [128]
tensor: dense_input_2/bias/.OPTIMIZER_SLOT/optimizer/m/.ATTRIBUTES/VARIABLE_VALUE (float32) [128]
tensor: dense_input_2/bias/.OPTIMIZER_SLOT/optimizer/v/.ATTRIBUTES/VARIABLE_VALUE (float32) [128]
tensor: dense_input_2/kernel/.ATTRIBUTES/VARIABLE_VALUE (float32) [778, 128]
tensor: dense_input_2/kernel/.OPTIMIZER_SLOT/optimizer/

## Training

In [47]:
model = ContextEncoder(final_dropout=0.5, 
                         dense_neurons=64, 
                         max_sentence_length=128, 
                         gamma=15)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [48]:
METRICS = [
      keras.metrics.BinaryAccuracy(name='accuracy')
]

In [49]:
EPOCHS = 5
BATCH_SIZE = 4

# balanced = balanced binary crossentropy
checkpoint_filepath = '../models/LDABERT2/simple/{}-{}-{}-pct-{}-aug/checkpoint.ckpt'.format(
                        dataset.dataset_type,                    
                        len(sentences), 
                        dataset.pct_data,
                        dataset.augment_pct)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    save_best_only=False,
    mode="auto",
    save_freq="epoch")

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

callbacks = [
#     early_stopping,
    model_checkpoint_callback
]

In [50]:
checkpoint_filepath

'../models/LDABERT2/simple/clinical-1622-0.1-pct-0.1-aug/checkpoint.ckpt'

In [51]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-3),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=METRICS)

In [52]:
try:
    model.load_weights(checkpoint_filepath)
except:
    print("No checkpoint available.")

No checkpoint available.


In [53]:
tf.expand_dims(tf.convert_to_tensor(saved_labels), axis=1)

<tf.Tensor: shape=(1642, 1), dtype=int32, numpy=
array([[1],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])>

In [54]:
history = model.fit([
                        left_input, mid_input, right_input, 
                        lda_left_input, lda_mid_input, lda_right_input
                    ], 
                    tf.convert_to_tensor(saved_labels), 
                    epochs=EPOCHS,
                    validation_split=0.1,
                    batch_size=BATCH_SIZE,
                    verbose=1, 
#                     class_weight=class_weight,
                    callbacks=callbacks)

Epoch 1/5
 14/370 [>.............................] - ETA: 27:35 - loss: 0.7415 - accuracy: 0.4788

KeyboardInterrupt: 

In [ ]:
from matplotlib import pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

## Experiment

In [14]:
from utils.experiments import get_experiments_json, get_experiments, save_results

In [15]:
experiments_config = get_experiments_json('ldabert_simple_test')
epxeriments_config_df = pd.DataFrame.from_dict(experiments_config)
epxeriments_config_df

,bert_type,dataset_type,final_dropout,dense_neurons,pct_data,augment_pct,epochs
0,ldabert,clinical,0.5,64,1,0.5,1000
1,ldabert,clinical,0.5,64,1,1.0,1000
2,ldabert,clinical,0.5,256,1,0.5,1000
3,ldabert,clinical,0.5,256,1,1.0,1000
4,ldabert,wiki,0.5,64,1,0.5,1000
5,ldabert,wiki,0.5,64,1,1.0,1000
6,ldabert,wiki,0.5,256,1,0.5,1000
7,ldabert,wiki,0.5,256,1,1.0,1000
8,ldabert,fiction,0.5,64,1,0.5,1000
9,ldabert,fiction,0.5,64,1,1.0,1000


In [8]:
# Read local `config.toml` file.
config = get_experiments('ALBERT_FINETUNE_SIMPLE')
config_df = pd.DataFrame.from_dict(config)
config_df

,bert_type,dataset_type,finetune_bert,pct_data,augment_pct,epochs
0,albert,clinical,True,1,0.1,200
1,albert,clinical,True,1,0.5,200
2,albert,clinical,True,1,1.0,200
3,albert,fiction,True,1,0.1,200
4,albert,fiction,True,1,0.5,200
5,albert,fiction,True,1,1.0,200
6,albert,wiki,True,1,0.1,200
7,albert,wiki,True,1,0.5,200
8,albert,wiki,True,1,1.0,200


In [69]:
config_df.to_csv(r'../models/experiment.csv', header=None, index=None, sep=' ', mode='a')

In [59]:
for experiment in config:
    bert_type = config['bert_type']
    dataset_type = config['dataset_type']
    finetune_bert = config['finetune_bert']
    pct_data = config['pct_data']
    augment_pct = config['augment_pct']
    epochs = config['epochs']
    print("params:", bert_type, dataset_type, finetune_bert, pct_data, augment_pct, epochs)
    
    # init model
    print("initializing model...")
    model = ContextEncoder(final_dropout=0.5,
                           dense_neurons=64,
                           bert_trainable=finetune_bert,
                           bert_type="albert-base-v2")
    
    # init dataset
    print("initializing dataset...")
    dataset = AlbertDataset(dataset_type=dataset_type,
                           pct_data=pct_data,
                           max_segment_length=5,
                           augment_pct=augment_pct)
    
    # process dataset
    print("processing dataset...")
    sentences, tokenized_sentences, labels = dataset.process()
    
    # create checkpoint path
    checkpoint_filepath = '../models/ALBERT/finetune/simple/{}-{}-{}-pct-{}-aug/checkpoint'.format(
                            dataset_type,                    
                            len(sentences), 
                            pct_data,
                            augment_pct)
    print(checkpoint_filepath)
    
    # compiling model
    print("compiling the model...")
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-3),
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=METRICS)
    
    try:
        model.load_weights(checkpoint_filepath)
        print("model loaded.")
    except:
        print("No checkpoint available.")
    
    # 
    print("starting the training process...")
    history = model.fit(dataset.format_sentences_tri_input(tokenized_sentences), 
                        tf.convert_to_tensor(labels), 
                        epochs=EPOCHS,
                        validation_split=0.1,
                        batch_size=BATCH_SIZE,
                        verbose=1, 
                        # class_weight=class_weight,
                        callbacks=callbacks)
    
    # assigning history to experiment object for saving.
    experiment["history"] = history
    
    print("saving results...")
    save_results(experiment)

9